In [249]:
import pandas as pd
import numpy as np

In [250]:
path = "../../../datos/Socioeconomicos/"
fname = "Consumo_electrico_2010_2017.csv"

df = ( pd.read_csv(path + fname)
    .drop( ["Consumo 2017", "Usuarios 2017"],
    axis = 1 ) )
df["Tarifa"] = df["Tarifa"].str.strip()
df.iloc[:,6:] = df.iloc[:,6:].abs()
df = df.query('CVE_MUN!=999 and CVE_ENT!=999 ')
df = df.query(
    'Tarifa=="1" or Tarifa=="1A" or Tarifa=="1B" '
    'or Tarifa=="1C" or Tarifa=="1D" or Tarifa=="1E" '
    'or Tarifa=="1F"'
    ' or Tarifa=="DAC"'
    )
df_2 = df.where(df["Tarifa"] != "DAC").dropna()

In [251]:
df_per = df.copy()
df_per["Consumo"] = ( df_per.iloc[:,6:13]
    .mean(axis = 1) )
df_per.drop(df_per.columns[6:13],
    axis = 1, inplace = True)
df_per["Usuarios"] = ( df_per.iloc[:,6:13]
    .mean(axis = 1) )
df_per.drop(df_per.columns[6:13],
    axis = 1, inplace = True)

df_sum = ( df_per.groupby("CVE_INEGI").sum()
    .drop(df_per.columns[0:2], axis = 1) )

for i in range(df_per.shape[0]):
    df_per.iloc[i, 6:] = ( df_per.iloc[i, 6:] /
        ( df_sum.iloc[df_sum.index == df_per.iloc[i,2]]
        .iloc[0] ) )

df_per2 = df_2.copy()
df_per2["Consumo"] = ( df_per2.iloc[:,6:13]
    .mean(axis = 1) )
df_per2.drop(df_per2.columns[6:13],
    axis = 1, inplace = True)
df_per2["Usuarios"] = ( df_per2.iloc[:,6:13]
    .mean(axis = 1) )
df_per2.drop(df_per2.columns[6:13],
    axis = 1, inplace = True)

df_sum2 = ( df_per2.groupby("CVE_INEGI").sum()
    .drop(df_per.columns[0:2], axis = 1) )

for i in range(df_per2.shape[0]):
    df_per2.iloc[i, 6:] = ( df_per2.iloc[i, 6:] /
        ( df_sum2.iloc[df_sum2.index == df_per2.iloc[i,2]]
        .iloc[0] ) )

In [258]:
df_eval = df_per2.copy()

umbral = 0.4387
#0.887
#0.4387

df_eval.iloc[:,6:] = df_per2.where(df_per2.iloc[:,6:] > umbral).iloc[:,6:]
df_eval.dropna(inplace = True)

df_eval.shape

(2469, 8)

In [259]:
df_dac = df_per[df_per["Tarifa"] == "DAC"].copy()

df_dac.sort_values("Consumo", ascending = False).iloc[0:85]#17

,CVE_ENT,CVE_MUN,CVE_INEGI,NOM_ENT,NOM_MUN,Tarifa,Consumo,Usuarios
44737,19,19,19019,Nuevo Leon ...,San Pedro Garza Garcia ...,DAC,0.416427,0.148404
12742,9,4,9004,Distrito Federal ...,Cuajimalpa de Morelos ...,DAC,0.398086,0.141936
8872,7,111,7111,Chiapas ...,Zinacantan ...,DAC,0.376670,0.106787
32137,15,37,15037,Mexico ...,Huixquilucan ...,DAC,0.351611,0.131892
13282,9,16,9016,Distrito Federal ...,Miguel Hidalgo ...,DAC,0.351123,0.103223
...,...,...,...,...,...,...,...,...
33217,15,61,15061,Mexico ...,Nopaltepec ...,DAC,0.106572,0.026086
31282,15,18,15018,Mexico ...,Calimaya ...,DAC,0.105786,0.024455
86722,26,2,26002,Sonora ...,Agua Prieta ...,DAC,0.104263,0.032902
23317,13,52,13052,Hidalgo ...,San Agustin Tlaxiaca ...,DAC,0.102808,0.021285


In [248]:
df_fin = df_2.copy()
df_fin = df_fin.groupby("CVE_INEGI").sum()

cols = ["CVE_ENT", "CVE_MUN", "NOM_ENT", "NOM_MUN"]

for i in cols:
    df_fin[i] = df.loc[df["Tarifa"] == "1", i].array

df_fin["Tarifa"] = df_eval["Tarifa"].array
df_fin = df_fin[ list(df_fin.columns[0:2])
    + list(df_fin.columns[-3:])
    + list(df_fin.columns[2:-3])
    ]
df_fin.reset_index(inplace = True)
df_fin = ( pd.concat( [df_fin, df[df["Tarifa"] == "DAC"]] )
    .sort_values([ "CVE_INEGI", "Tarifa"] ) )

df_fin.to_csv(path + "Electrico_residencial_2010_2017.csv",
    index = False)